# Computing the nonadiabatic couplings in Kohn-Sham and excited states bases

In this tutorial, we will start computing the nonadiabatic couplings (NACs) from the molecular orbital overlap files obtained in [step2](../../7_step2_cp2k/1_DFT). The NACs will be computed in Kohn-Sham states and also in both single-partcile (SP) and many-body (MB) excited state bases. Finally we will plot the excited states energies vs time and NAC map.

## Table of contents
<a name="toc"></a>
1. [Importing needed libraries](#import)\
2. [Overview of required files](#required_files)\
3. [Computing the NACs](#comp_nacs)     \
   3.1. [Kohn-Sham basis](#KS)\
   3.2. [Excited state basis](#excited_states)
   

### A. Learning objectives

* To be able to compute the NACs in Kohn-Sham and excited state bases

### B. Use cases

* [Computing the NACs](#comp_nacs)


### C. Functions

- `libra_py`
  - `workflows`
    - `nbra`
      - [`step3`](#comp_nacs)
        - [`run_step3_ks_nacs_libint`](#KS)
        - [`run_step3_sd_nacs_libint`](#excited_states)


## 1. Importing needed libraries <a name="import"></a>
[Back to TOC](#toc)

Since the data are stored in sparse format using `scipy.sparse` library, we need to load this library so that we can read and check the orthonormality of the data.
Import `numpy`, `scipy.sparse`, `CP2K_methods`, `data_stat`, `data_io`, `units`, `step2_many_body`, and `step3` modules. Also, `glob` will be needed to find specific types of files.

In [1]:
import os
import sys
import time
import glob
import math
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from liblibra_core import *
from libra_py.workflows.nbra import step3

/projects/academic/alexeyak/mohammad/software/mc/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/alexeyak/mohammad/software/mc/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/projects/academic/alexeyak/mohammad/software/mc/envs/libra/lib/python3.7/importlib/_bootstrap.py:219: R

## 2. Overview of required files <a name="required_files"></a>
[Back to TOC](#toc)

* `../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/res`

The MO overlap files are needed and stored in this folder.

* `../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/all_logfiles`

All of the logfiles obtained from the electronic structure calculations of CP2K. These files will be needed to find the Kohn-Sham HOMO index and also extract the the TD-DFT data for each step.

## 3. Computing the NACs <a name="comp_nacs"></a>
[Back to TOC](#toc)

### 3.1. Kohn-Sham basis <a name="KS"></a>

The `libra_py.workflow.nbra.step3.run_step3_ks_nacs_libint(params)` computes the NACs between pairs of Kohn-Sham states using the molecular orbital overlaps. 

Libra stores the overlap data in 2-spinor format as follows:

<div>
<img src="./active_space_0.png" width="500"/>
</div>

With no spin-orbit couplings, two blocks of the matrix is zero. Since most of the elements of the overlap matrices are zero, we use the `scipy.sparse` library for storing and loading them. 

The paramters for this function are as follows:

`params['lowest_orbital']`: The lowest orbital considered in the computation of the MO overlaps. This value is exactly the same
as in the `run_template.py` file in step2.

`params['highest_orbital']`: The highest orbital considered in the computation of the MO overlaps. This value is exactly the same
as in the `run_template.py` file in step2.

`params['num_occ_states']`: The number of occupied orbitals to be considered from HOMO to lower occupied states. This value is defined by user.

`params['num_unocc_states']`: The number of unoccupied orbitals to be considered from LUMO to higher unoccupied states. This value is defined by user.

The two values above are used to create an active space which then will be used to select the elements from the MO overlap and 
energy matrices. 

`params['use_multiprocessing']`: A boolean flag to use the multiprocessing library of Python or not.

`params['nprocs']`: The number of processors to be used for the calculations. Libra will use this only if the `params['use_multiprocessing']` 
is set to `True`.

`params['time_step']`: The time-step used in the calculations in `fs`.

`params['es_software']`: The name of the software package used to compute the electronic structure calculations. This will be used to generate the HOMO 
index of that system so it can build the active space.

`params['path_to_npz_files']`: The full path to the MO overlap files.

`params['logfile_directory']`: The full path to the folder where all the log files are stored.

`params['path_to_save_ks_Hvibs']`: The full path to the folder in which the NACs between the Konh-Sham states are stored.

`params['start_time']`: The start time-step.

`params['finish_time']`: The finish time-step.

`params['apply_phase_correction']`: A boolean flag for applying phase-correction algorithm.

`params['apply_orthonormalization']`: A boolean flag for applying the orthonormalization algorithm.

`params['do_state_reordering']`: If this value is set to `1` or `2`, the state-reordering will be applied to overlap matrices.

`params['state_reordering_alpha']`: The state-reordering alpha value if the `params['do_state_reordering'] = 2`.

`params['nac_algo']`: selection of a method to compute NACs. `0` is for Hamess-Shiffer-Tully (HST), finite difference (the default) and `1` is Meek-Levine NPI approach.

After setting all the above paramters, the calculations are run using `step3.run_step3_ks_nacs_libint(params)`.


In fact, we generate a new active space from the raw data we obtained from step2, and make a new set of 
overlap matrices that some corrections are applied such as state reordering and phase-correction. Then, one can use the 
corrected overlaps for the computation of the NACs between excited states, using the `step3.run_step3_sd_nacs_libint` function. 
A schematic of the workflow for selecting new active space is shown below:

<div>
<img src="./active_space_1.png" width="1000"/>
</div>


Note: When creating active spaces, the prior knowledge of the KS HOMO index by user is advantageous.

In [2]:
params_ks = {
              'lowest_orbital': 24-10, 'highest_orbital': 24+11, 'num_occ_states': 10, 'num_unocc_states': 10,
              'use_multiprocessing': True, 'nprocs': 8, 'time_step': 1.0, 'es_software': 'cp2k',
              'path_to_npz_files': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/res',
              'logfile_directory': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/all_logfiles',
              'path_to_save_ks_Hvibs': os.getcwd()+'/res-ks-DFT',
              'start_time': 1200, 'finish_time': 1401,
            }

#### For KS states - Applying correction to KS overlaps and computing the NACs in KS space
step3.run_step3_ks_nacs_libint(params_ks)

mkdir: cannot create directory ‘/projects/academic/alexeyak/mohammad/my_repositories/Tutorials_Libra/6_dynamics/2_nbra_workflows/8_step3/1_DFT/res-ks-DFT’: File exists


Computing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in step Computing orthonormalization of St matrices in stepComputing orthonormalization of St matrices in step    1221   1207121412001235
1228
12421249





Done with step Done with stepDone with stepDone with step1242Done with step   Done with step  12211207 122812491200   . Elapsed time:. Elapsed time:   . Elapsed time: 1.699648380279541 1.7428374290466309Done with step
Computing orthonormalization of St matrices in step. Elapsed time:
 1.7278804779052734. Elapsed time: 1.7447142601013184 Computing orthonormalization of St matrices in step

. Elapsed time:  1.7639572620391846
Computing orthonormalization of St matrices in step 1229
Computing orthonormalization of St matrices in step1208 123

 . Elapsed time: 2.0053870677948
Computing orthonormalization of St matrices in step 1272
Done with step 1292 . Elapsed time: 2.0223402976989746
Computing orthonormalization of St matrices in step 1293
Done with step 1306Done with step  . Elapsed time: 12582.0635664463043213 . Elapsed time:
 Computing orthonormalization of St matrices in step 13071.5179214477539062

Computing orthonormalization of St matrices in step 1259
Done with step 1279 . Elapsed time: 1.7149558067321777
Computing orthonormalization of St matrices in step 1280
Done with step 1300 . Elapsed time: 1.6344645023345947
Computing orthonormalization of St matrices in step 1301Done with step 1293 . Elapsed time: 
1.492793083190918Done with step 
Computing orthonormalization of St matrices in step 1294
1265 . Elapsed time: 1.8711817264556885
Done with step Done with stepComputing orthonormalization of St matrices in step 1272  1266. Elapsed time: 1286 Done with step
1.8933758735656738. Elapsed time: 2.0232131481170654
Comp

Computing orthonormalization of St matrices in step  1317
1.5153357982635498 
1.3848748207092285Computing orthonormalization of St matrices in step
Computing orthonormalization of St matrices in step 1365 
1358
Done with step 1350 . Elapsed time: 1.9266648292541504
Computing orthonormalization of St matrices in step 1351
Done with step Done with step 1337 Done with step. Elapsed time:1344  . Elapsed time: 13301.8503351211547852 Done with step  1323 
2.123483896255493. Elapsed time: Done with step1.7932376861572266
Computing orthonormalization of St matrices in step. Elapsed time:Computing orthonormalization of St matrices in step
 1.7087373733520508 1324
 Computing orthonormalization of St matrices in step 
13171338  . Elapsed time:
1345 Computing orthonormalization of St matrices in step
 1.5960159301757812
Computing orthonormalization of St matrices in step 1318
1331
Done with step 1358 Done with step 1365. Elapsed time:  1.7396976947784424
. Elapsed time: Computing orthonormalizatio

### 3.2. Excited state basis <a name="excited_states"></a>
[Back to TOC](#toc)

Below, we will be using `step3.run_step3_sd_nacs_libint` function to compute the time-overlaps and nonadiabatic couplings between excited states basis. This can be done either in the many-body or single-particle basis. A schematic of the workflow for selecting new active space is shown below:


Some parameters are common with the ones used to run `step3.run_step3_ks_nacs_libint(params)` above.

Other parameters needed to run the `step3.run_step3_sd_nacs_libint(params)` function are as follows:

`params['isUKS']`: A boolean flag for unrestricted spin calculations.

`params['is_many_body']`: If set to `True`, the NACs will be computed between pairs of many-body (TD-DFT) states. Also, the NACs between single-particle 
SDs obtained from the TD-DFT results will be computed as well. Otherwise, only single-particle NACs will be computed only for the SDs obtained from
`num_occ_states` and `num_unocc_states`. This will be used for xTB calculations in which no TD-DFT is performed.

`params['number_of_states']`: The number of TD-DFT states to consider. This value should not exceed the number of requested TD-DFT states in the CP2K
calculations.

`params['tolerance']`: A lower bound for selection of the excitation with configuration interaction coefficients higher than this value.

`params['verbosity']`: An integer value showing the printing level. The default is set to 0. Higher values will print more data on the terminal.

`params['sorting_type']`: After defining the SDs, Libra will sort them either based on `'energy'` or `'identity'`.


The NACs can also be computed between excited states. These include the single-particle and many-body bases which the latter is obtained from the
TD-DFT calculations. First, we need to compute the overlap between excited state Slater-determinants (SDs) then they will be used to compute the NACs
between them. For many-body states, the configuration interaction coefficietns will also be used. We will consider both single-particle 
and many-body for DFT calculations but only single-particle for xTB.

There are different ways of defining the excited states SDs (the single-particle excited state basis). The first is through 
defining the `num_occ_states` and `num_unocc_states` in which Libra
will start making the SDs from all of the occupied states (starting from `HOMO-num_occ_states+1`) to all of the unoccupied states (ends
to `LUMO+num_unocc_states-1`). Also, if the unrestricted spin calculation flag is set to `True`, the SDs will be made for both alpha and beta spin channels. 

For example, if you want to build the electron-only excitation basis, you need to set `params['num_occ_states'] = 1` and set `params['num_unocc_states']`
to a value less than the number of unoccupied orbitals that was considered in the computation of overlaps. This will generate all the electron-only
excitation from HOMO to unoccupied states.

If the TD-DFT calculations has been done, then Libra will go over all log files and 
generate all the SDs used for all the steps and therefore the definition of these SDs is automatic and Libra will replace the `num_occ_states` and
`num_unocc_states` itself based on the SDs that were generated from the TD-DFT log files. 



In [3]:
params_mb_sd = {
          'lowest_orbital': 24-10, 'highest_orbital': 24+11, 'num_occ_states': 10, 'num_unocc_states': 10,
          'isUKS': 0, 'number_of_states': 10, 'tolerance': 0.01, 'verbosity': 0, 'use_multiprocessing': True, 'nprocs': 12,
          'is_many_body': True, 'time_step': 1.0, 'es_software': 'cp2k',
          'path_to_npz_files': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/res',
          'logfile_directory': os.getcwd()+'/../../7_step2_cp2k/1_DFT/2_hpc/1_example_TiO2/all_logfiles',
          'path_to_save_sd_Hvibs': os.getcwd()+'/res-mb-sd-DFT',
          'outdir': os.getcwd()+'/res-mb-sd-DFT', 'start_time': 1200, 'finish_time': 1401, 'sorting_type': 'identity',
         }

step3.run_step3_sd_nacs_libint(params_mb_sd)

mkdir: cannot create directory ‘/projects/academic/alexeyak/mohammad/my_repositories/Tutorials_Libra/6_dynamics/2_nbra_workflows/8_step3/1_DFT/res-mb-sd-DFT’: File exists


sd_unique_basis is: [[[24, 25], 'alp'], [[24, 26], 'alp'], [[23, 25], 'alp'], [[22, 25], 'alp'], [[21, 25], 'alp'], [[23, 26], 'alp'], [[22, 26], 'alp'], [[24, 27], 'alp'], [[24, 29], 'alp'], [[21, 26], 'alp'], [[20, 25], 'alp'], [[24, 28], 'alp'], [[20, 26], 'alp'], [[21, 30], 'alp'], [[19, 26], 'alp'], [[19, 25], 'alp'], [[20, 27], 'alp'], [[23, 27], 'alp'], [[23, 28], 'alp'], [[24, 30], 'alp']]
sd_states_reindexed is: [[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 7, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 8, -6], [1, -1, 2, -2, 3, -3, 4, -4, 7, -5, 6, -6], [1, -1, 2, -2, 3, -3, 7, -4, 5, -5, 6, -6], [1, -1, 2, -2, 7, -3, 4, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 8, -5, 6, -6], [1, -1, 2, -2, 3, -3, 8, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 9, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 11, -6], [1, -1, 2, -2, 8, -3, 4, -4, 5, -5, 6, -6], [1, -1, 7, -2, 3, -3, 4, -4, 5, -5, 6, -6], [1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 10, -6], 

Done with computing the SD overlap of step 24. Elapsed time 0.02365398406982422Computing the SD overlaps for stepComputing the SD overlaps for step  8075
70Done with computing the SD overlap of step 54. Elapsed time 0.021085739135742188

Done with computing the SD overlap of step 44. Elapsed time 0.021016836166381836

Done with computing the SD overlap of step 39. Elapsed time 0.021175622940063477Computing the SD overlaps for stepDone with computing the SD overlap of step 29. Elapsed time 0.02457141876220703 
Computing the SD overlaps for step
85

 Computing the SD overlaps for stepComputing the SD overlaps for stepDone with computing the SD overlap of step 19. Elapsed time 0.021999835968017578Computing the SD overlaps for step  95 
10590


100Computing the SD overlaps for step
 110
Done with computing the SD overlap of step 59. Elapsed time 0.022592544555664062
Computing the SD overlaps for step 115
Done with computing the SD overlap of step 60. Elapsed time 0.021345138549804688Done w

Done with computing the SD overlap of step 155. Elapsed time 0.02201223373413086121Done with computing the SD overlap of step 140. Elapsed time 0.021633386611938477Done with computing the SD overlap of step 145. Elapsed time 0.02114582061767578Done with computing the SD overlap of step 135. Elapsed time 0.0208587646484375Computing the SD overlaps for step



Done with computing the SD overlap of step 150. Elapsed time 0.020925045013427734Computing the SD overlaps for stepDone with computing the SD overlap of step 160. Elapsed time 0.02147650718688965 
Computing the SD overlaps for step
131Done with computing the SD overlap of step 125. Elapsed time 0.020884037017822266Computing the SD overlaps for stepComputing the SD overlaps for step


 Computing the SD overlaps for step Computing the SD overlaps for step   151Computing the SD overlaps for step126Done with computing the SD overlap of step 170. Elapsed time 0.020890474319458008
156161146 



136 Done with computing the SD overlap of s

Done with computing the SD overlap of step 192. Elapsed time 0.03335714340209961
Computing the SD overlaps for step 193
Done with computing the SD overlap of step 183. Elapsed time 0.02006673812866211
Computing the SD overlaps for step 184
Done with computing the SD overlap of step 188. Elapsed time 0.019352436065673828
Computing the SD overlaps for step 189
Done with computing the SD overlap of step 198. Elapsed time 0.020202159881591797
Computing the SD overlaps for step 199
Done with computing the SD overlap of step 193. Elapsed time 0.025331735610961914Done with computing the SD overlap of step 184. Elapsed time 0.018571138381958008

Computing the SD overlaps for step 194
Done with computing the SD overlap of step 189. Elapsed time 0.02155447006225586Done with computing the SD overlap of step 199. Elapsed time 0.020346641540527344

Done with computing the SD overlap of step 194. Elapsed time 0.01862645149230957
Done with computing the SD overlaps. Elapsed time: 1.2774105072021484
